## Importando as bibliotecas 

Nesta etapa são importadas e baixadas as bibliotecas e pacotes das mesmas, necessárias para as análises, gráficos, funções e para gerar o modelo Bag of Words.

In [1]:
pip install openpyxl



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/250.0 kB ? eta -:--:--
     -------------------------------------  245.8/250.0 kB 7.4 MB/s eta 0:00:01
     -------------------------------------- 250.0/250.0 kB 7.5 MB/s eta 0:00:00


In [5]:
pip install scikit-learn

                                              0.0/8.3 MB ? eta -:--:--
                                              0.2/8.3 MB 5.3 MB/s eta 0:00:02
     -                                        0.2/8.3 MB 4.8 MB/s eta 0:00:02
     ----                                     0.9/8.3 MB 7.4 MB/s eta 0:00:01
     -------                                  1.5/8.3 MB 8.5 MB/s eta 0:00:01
     ---------                                2.0/8.3 MB 10.0 MB/s eta 0:00:01
     ------------                             2.6/8.3 MB 10.4 MB/s eta 0:00:01
     ----------------                         3.4/8.3 MB 11.3 MB/s eta 0:00:01
     ------------------                       3.8/8.3 MB 10.9 MB/s eta 0:00:01
     ---------------------                    4.5/8.3 MB 11.6 MB/s eta 0:00:01
     -------------------------                5.2/8.3 MB 12.4 MB/s eta 0:00:01
     ---------------------------              5.7/8.3 MB 11.8 MB/s eta 0:00:01
     --------------------------------         6.7/8.3 MB 12.7 MB


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: keras in c:\users\eduar\appdata\local\programs\python\python311\lib\site-packages (2.12.0)




[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
 pip install tensorflow

  Using cached tensorflow-2.12.0-cp311-cp311-win_amd64.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import nltk 
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
nltk.download('punkt') 
nltk.download('stopwords') 
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eduar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Processamento dos dados 

In [26]:
# Abrindo a base de dados 
df = pd.read_excel('../base/base.xlsx', index_col=0)

### Pré processamento dos dados 

Nesta etapa o foco é limpar da base de dados os dados que não serão utilizados.
Portanto, com essa função são removidos todos os comentários que tenham autoria do BTG Pactual e fotos de terceiros que marcavam a companhia.


In [16]:
# Esta função remove os dados que não serão utilizados para a análise 
def cleanData(df):
    # Remove as aspas dos nomes das colunas
    df = df.rename(columns=lambda x: x.replace('"', ''))
    # Remove comentários feito pelo btg
    dfWithoutAutor = df[df["autor"] != "btgpactual"]
    # Remove comentários nulos
    dfWithoutNull = dfWithoutAutor[dfWithoutAutor["autor"].notnull()]
    # Deixando apenas as interações do tipo comentário e resposta
    dfFinal = dfWithoutNull.loc[(dfWithoutNull['tipoInteracao'] == 'comentário') | (dfWithoutNull['tipoInteracao'] == 'resposta')]
    return dfFinal
    

### Processamento dos dados

Nesta etapa as frases são tokenizadas (cada palavra da frase é dividida, sendo 1 palavra = 1 token) e removidas as stop words, palavras que não acrescentam significado semântico relevante (e.g: de, para, ou). Além disso, é gerado um arquivo .csv com uma nova coluna que contém os tratamento citados acima aplicado aos comentários (coluna 'texto'). 


In [17]:
def preprocessing_coments(frase):
    palavras = frase.split() #divide as frases em palavras
    filtered_words = [word for word in palavras if word.lower() not in stop_words and not word.startswith('@') and word.isalpha()] #coloca todas as letras em minúsculas (lower) e remove as stop words
    tokens = word_tokenize(" ".join(filtered_words)) #tokeniza as palavras e organiza cada comentário em uma lista de palavras
    return tokens

### Aplicação do modelo

Durante esta etapa é feita a implementação do modelo Bag of Words, que vetoriza as palavras e faz a contagem de vezes que cada uma aparece na coluna a ser utilizada. Assim, tendo como resultado um dataframe com as palavras que mais aparecem no texto e sua respectiva frequência.

In [24]:
def bow_dataframe(inputUser):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(inputUser)
    wordCount = tokenizer.word_counts    # Com essa variavel é possível ver as palavras e contar sua frequência
    dfCountBoW = pd.DataFrame(list(wordCount.items()))
    dfCountBoW.rename(columns={0: "Palavra", 1:"Frequência"}, inplace=True)
    final_df = dfCountBoW.sort_values(by=['Frequência'], ascending=False)
    return final_df

### Resultado bag of words

Visualização dos resultados do Bag of Words

In [20]:
base_nova = bag_of_words(df)

In [22]:
base_nova.head(7)


,Palavra,Frequência
97,banco,475
6,btg,408
66,pra,386
590,limite,363
16,conta,319
299,cartão,310
188,melhor,228


### Pipeline de processamento completo 

A função abaixo 

In [25]:

#Função para rodar nosso modelo de bag of words, essa função recebe um dataframe
def bag_of_words(base):
    #limpeza da base de dados
    base1 = cleanData(base)
    #tokenização da coluna de textos
    base1['Frases_sem_stop_words'] = base1['texto'].apply(preprocessing_coments)
    base1 = bow_dataframe(base1['Frases_sem_stop_words'])
    return base1
    